In [8]:
import sys, os
import numpy as np
import pandas as pd
import random
from random import shuffle, choice
import time
import os
import glob
import keras
import tensorflow as tf
from keras.utils import np_utils
from keras.models import Sequential
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras import backend as K
from tensorflow.keras.layers import *
from tensorflow.keras.optimizers import *
from tensorflow.keras.models import load_model
from tensorflow.keras import regularizers
from random import shuffle, choice
from sklearn.preprocessing import MinMaxScaler
import sklearn.metrics as metrics
from sklearn.metrics import log_loss
from sklearn.metrics import confusion_matrix
from keras.callbacks import ModelCheckpoint
from keras.models import Model
import matplotlib.pyplot as plt
from sklearn.preprocessing import normalize
from keras.models import load_model
from sklearn.preprocessing import MinMaxScaler,StandardScaler

batch_size = 250
epochs = 100
epochs_traits = 500
num_classes = 4

#From: https://towardsdatascience.com/neural-networks-ensemble-33f33bea7df3
class LinearW(Layer):
    def __init__(self):
        super(LinearW, self).__init__()
    def build(self, input_shape):
        self.W = self.add_weight(name='name',
                    shape=(1,1,len(input_shape)),
                    initializer='uniform',
                    dtype=tf.float32,
                    trainable=True)     
    def call(self, inputs):
        # inputs is a list of tensor of shape [(n_batch, n_feat), ..., (n_batch, n_feat)]
        # expand last dim of each input passed [(n_batch, n_feat, 1), ..., (n_batch, n_feat, 1)]
        inputs = [tf.expand_dims(i, -1) for i in inputs]
        inputs = Concatenate(axis=-1)(inputs) # (n_batch, n_feat, n_inputs)
        weights = tf.nn.softmax(self.W, axis=-1) # (1,1,n_inputs)
        # weights sum up to one on last dim
        return tf.reduce_sum(weights*inputs, axis=-1) # (n_batch, n_feat)


def create_mlp(traitstrain, regularizer=None):
  """Creates a three-layer MLP with inputs of the given dimension"""
  model = Sequential()
  model.add(Dense(150, use_bias=False, input_dim=traitstrain.shape[1], activation="relu", kernel_regularizer=regularizers.l1(0.001)))
  model.add(BatchNormalization())
  #model.add(Dropout(0.5))
  model.add(Dense(150, use_bias=False, activation="relu"))
  model.add(BatchNormalization())
  #model.add(Dropout(0.5))
  model.add(Dense(50, activation="relu"))
  return model


def create_cnn(xtest, regularizer=None):
  inputShape = (xtest.shape[1], xtest.shape[2])
  inputs = Input(shape=inputShape)
  x = inputs
  x = Conv1D(250, kernel_size=3, activation='relu', use_bias=False, input_shape=(xtest.shape[1], xtest.shape[2]))(x)
  x = BatchNormalization()(x)
  x = Conv1D(250, kernel_size=3, use_bias=False, activation='relu')(x)
  x = BatchNormalization()(x)
  x = Conv1D(250, kernel_size=3, use_bias=False, activation='relu')(x)
  x = BatchNormalization()(x)
  x = MaxPooling1D(pool_size=3)(x)
  x = Flatten()(x)
  x = Dense(125, activation='relu')(x)
  x = Dropout(0.5)(x)
  x = Dense(125, activation='relu')(x)
  x = Dropout(0.5)(x)
  # Apply another fully-connected layer, this one to match the number of nodes coming out of the MLP
  x = Dense(50, kernel_regularizer=regularizer)(x)
  x = Activation("relu")(x)
  # Construct the CNN
  model = Model(inputs, x)
  # Return the CNN
  return model

In [9]:
################################################################################################################################################
#Load data
################################################################################################################################################

traits_BM = []
traits_BM = np.loadtxt("./traits/traits_BM.txt").reshape(40000,-1,4)
traits_BM = np.array(traits_BM)


scalers_BM = {}
for i in range(traits_BM.shape[2]):
    scalers_BM[i] = StandardScaler(copy=False)
    traits_BM[:, :, i] = scalers_BM[i].fit_transform(traits_BM[:, :, i]) 

traits_BM = np.array(traits_BM)


traits_OU = []
traits_OU = np.loadtxt("./traits/traits_OU.txt").reshape(40000,-1,4)

traits_OU = np.array(traits_OU)

scalers_OU = {}
for i in range(traits_OU.shape[2]):
    scalers_OU[i] = StandardScaler(copy=False)
    traits_OU[:, :, i] = scalers_OU[i].fit_transform(traits_OU[:, :, i]) 

traits_OU = np.array(traits_OU)

#Add missing individuals in E. balsamifera subsp. balsamifera as 0s
for i in range(traits_BM.shape[0]):
  for m in range(51):
    j = random.randint(19, 98)
    traits_BM[i,j,:] = 0
    
#Add missing individuals in E. balsamifera subsp. balsamifera as 0s
for i in range(traits_OU.shape[0]):
  for m in range(51):
    j = random.randint(19, 98)
    traits_OU[i,j,:] = 0

u1 = np.load("./trainingSims/Model_1sp.npz")
u2 = np.load("./trainingSims/Model_2spMorph.npz")
u3 = np.load("./trainingSims/Model_2spPhylo.npz")
u4 = np.load("./trainingSims/Model_3sp.npz")

x=np.concatenate((u1['Model_1sp'],u2['Model_2spMorph'],u3['Model_2spPhylo'],u4['Model_3sp']),axis=0)

#transform major alleles in -1 and minor 1
for arr,array in enumerate(x):
  for idx,row in enumerate(array):
    if np.count_nonzero(row) > len(row)/2:
      x[arr][idx][x[arr][idx] == 1] = -1
      x[arr][idx][x[arr][idx] == 0] = 1
    else:
      x[arr][idx][x[arr][idx] == 0] = -1

y=[0 for i in range(len(u1['Model_1sp']))]
y.extend([1 for i in range(len(u2['Model_2spMorph']))])
y.extend([2 for i in range(len(u3['Model_2spPhylo']))])
y.extend([3 for i in range(len(u4['Model_3sp']))])
y = np.array(y)

print (len(x), len(y), len(traits_BM))
shf = list(range(len(x)))
shuffle(shf)

y = y[shf]
x = x[shf]
traits_BM = traits_BM[shf]
traits_OU = traits_OU[shf]

#Add missing data as 0s, according to a specifies missing data percentage
#46,761 SNP datapoints and 8,287 missing genotypes = 17.7%
missD_perc = 17.7
missD = int(x.shape[1]*x.shape[2]*(missD_perc/100))
for i in range(x.shape[0]):
  for m in range(missD):
    j = random.randint(0, x.shape[1] - 1)
    k = random.randint(0, x.shape[2] - 1)
    x[i][j][k] = 0

del(missD)

xtrain, xtest = x[int(len(y)*.25):], x[:int(len(y)*.25)]
ytrain, ytest = y[int(len(y)*.25):], y[:int(len(y)*.25)]
traits_train, traits_test = traits_BM[int(len(y)*.25):], traits_BM[:int(len(y)*.25)]


ytest = np_utils.to_categorical(ytest, num_classes)
ytrain = np_utils.to_categorical(ytrain, num_classes)
traits_train=traits_train.reshape((traits_train.shape[0], (traits_train.shape[1]*traits_train.shape[2])))
traits_test=traits_test.reshape((traits_test.shape[0], (traits_test.shape[1]*traits_test.shape[2])))

# Create the MLP and CNN models
mlp = create_mlp(traits_train)
cnn = create_cnn(xtest)

40000 40000 40000


In [3]:
################################################################################################################################################
#Train the combined SNPs + traits (BM) network
################################################################################################################################################

# Create the input to the final set of layers as the output of both the MLP and CNN
combinedInput = LinearW()([mlp.output, cnn.output])

# The final fully-connected layer head will have two dense layers (one relu and one sigmoid)
x = Dense(50, activation="relu")(combinedInput)
x = Dense(num_classes, activation="softmax")(x)


# The final model accepts numerical data on the MLP input and images on the CNN input, outputting a single value
model1 = Model(inputs=[mlp.input, cnn.input], outputs=x)

opt = SGD(learning_rate=0.001)

model1.compile(loss=keras.losses.categorical_crossentropy,
	              optimizer=opt,
	              metrics=['accuracy'])

print(model1.summary())

earlyStopping = EarlyStopping(monitor='val_accuracy', patience=100, verbose=0, mode='max', restore_best_weights=True)
start = time.time()
model1.fit([traits_train, xtrain], ytrain, batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=([traits_test, xtest], ytest),callbacks=[earlyStopping])
print (f'Time: {time.time() - start}')

model1.save(filepath='./TrainedModels/Trained_Comb_Model_1KSNPs_BM.acc.mod')

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 429, 109)]   0           []                               
                                                                                                  
 conv1d (Conv1D)                (None, 427, 250)     81750       ['input_1[0][0]']                
                                                                                                  
 batch_normalization_2 (BatchNo  (None, 427, 250)    1000        ['conv1d[0][0]']                 
 rmalization)                                                                                     
                                                                                                  
 conv1d_1 (Conv1D)              (None, 425, 250)     187500      ['batch_normalization_2[0][

2023-03-22 16:15:29.627586: I tensorflow/stream_executor/cuda/cuda_blas.cc:1614] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
2023-03-22 16:15:29.796505: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8401
2023-03-22 16:15:31.286497: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2023-03-22 16:15:31.288258: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2023-03-22 16:15:31.288273: W tensorflow/stream_executor/gpu/asm_compiler.cc:80] Couldn't get ptxas version string: INTERNAL: Couldn't invoke ptxas --version
2023-03-22 16:15:31.289946: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2023-03-22 16:15:31.290010: W tensorflow/stream_executor/gpu/redzone_allocator.cc:314] INTERNAL: Failed to launch ptxas
Relying on driver to

120/120 [==============================] - 12s 60ms/step - loss: 4.7626 - accuracy: 0.2998 - val_loss: 4.6758 - val_accuracy: 0.3411
Epoch 2/100
120/120 [==============================] - 6s 51ms/step - loss: 4.5858 - accuracy: 0.4047 - val_loss: 4.5034 - val_accuracy: 0.5343
Epoch 3/100
120/120 [==============================] - 6s 50ms/step - loss: 4.3895 - accuracy: 0.5185 - val_loss: 4.2025 - val_accuracy: 0.7027
Epoch 4/100
120/120 [==============================] - 6s 51ms/step - loss: 4.2055 - accuracy: 0.6048 - val_loss: 3.9548 - val_accuracy: 0.8206
Epoch 5/100
120/120 [==============================] - 7s 55ms/step - loss: 4.0680 - accuracy: 0.6660 - val_loss: 3.7832 - val_accuracy: 0.8745
Epoch 6/100
120/120 [==============================] - 6s 51ms/step - loss: 3.9486 - accuracy: 0.7158 - val_loss: 3.6587 - val_accuracy: 0.9022
Epoch 7/100
120/120 [==============================] - 6s 53ms/step - loss: 3.8539 - accuracy: 0.7602 - val_loss: 3.5682 - val_accuracy: 0.9274
Epo

INFO:tensorflow:Assets written to: ./TrainedModels/Trained_Comb_Model_1KSNPs_BM.acc.mod/assets


INFO:tensorflow:Assets written to: ./TrainedModels/Trained_Comb_Model_1KSNPs_BM.acc.mod/assets


In [4]:
################################################################################################################################################
#Train the SNPs only network
################################################################################################################################################

xCNN = Dense(num_classes, activation="softmax")(cnn.output)

model2 = Model(inputs=cnn.input, outputs=xCNN)

model2.compile(loss=keras.losses.categorical_crossentropy,
	              optimizer=opt,
	              metrics=['accuracy'])

print(model2.summary())

start = time.time()
model2.fit(xtrain, ytrain, batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(xtest, ytest),callbacks=[earlyStopping])
print (f'Time: {time.time() - start}')

model2.save(filepath='./TrainedModels/Trained_CNN_Model_1KSNPs.acc.mod')

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 429, 109)]        0         
                                                                 
 conv1d (Conv1D)             (None, 427, 250)          81750     
                                                                 
 batch_normalization_2 (Batc  (None, 427, 250)         1000      
 hNormalization)                                                 
                                                                 
 conv1d_1 (Conv1D)           (None, 425, 250)          187500    
                                                                 
 batch_normalization_3 (Batc  (None, 425, 250)         1000      
 hNormalization)                                                 
                                                                 
 conv1d_2 (Conv1D)           (None, 423, 250)          1875

120/120 [==============================] - 6s 51ms/step - loss: 0.0130 - accuracy: 0.9965 - val_loss: 0.0049 - val_accuracy: 0.9986
Epoch 95/100
120/120 [==============================] - 6s 53ms/step - loss: 0.0120 - accuracy: 0.9965 - val_loss: 0.0050 - val_accuracy: 0.9986
Epoch 96/100
120/120 [==============================] - 6s 51ms/step - loss: 0.0131 - accuracy: 0.9959 - val_loss: 0.0064 - val_accuracy: 0.9985
Epoch 97/100
120/120 [==============================] - 6s 51ms/step - loss: 0.0132 - accuracy: 0.9965 - val_loss: 0.0058 - val_accuracy: 0.9984
Epoch 98/100
120/120 [==============================] - 6s 50ms/step - loss: 0.0104 - accuracy: 0.9967 - val_loss: 0.0044 - val_accuracy: 0.9986
Epoch 99/100
120/120 [==============================] - 6s 49ms/step - loss: 0.0101 - accuracy: 0.9974 - val_loss: 0.0059 - val_accuracy: 0.9985
Epoch 100/100
120/120 [==============================] - 6s 50ms/step - loss: 0.0122 - accuracy: 0.9964 - val_loss: 0.0051 - val_accuracy: 0.99

INFO:tensorflow:Assets written to: ./TrainedModels/Trained_CNN_Model_1KSNPs.acc.mod/assets


INFO:tensorflow:Assets written to: ./TrainedModels/Trained_CNN_Model_1KSNPs.acc.mod/assets


In [5]:
################################################################################################################################################
#Train the traits only (BM) network
################################################################################################################################################

xMLP = Dense(num_classes, activation="softmax")(mlp.output)

model3 = Model(inputs=mlp.input, outputs=xMLP)

model3.compile(loss=keras.losses.categorical_crossentropy,
	              optimizer=opt,
	              metrics=['accuracy'])

print(model3.summary())

start = time.time()
model3.fit(traits_train, ytrain, batch_size=batch_size,
          epochs=epochs_traits,
          verbose=1,
          validation_data=(traits_test, ytest),callbacks=[earlyStopping])
print (f'Time: {time.time() - start}')
          
model3.save(filepath='./TrainedModels/Trained_Traits_Model_BM.acc.mod')

Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_input (InputLayer)    [(None, 436)]             0         
                                                                 
 dense (Dense)               (None, 150)               65400     
                                                                 
 batch_normalization (BatchN  (None, 150)              600       
 ormalization)                                                   
                                                                 
 dense_1 (Dense)             (None, 150)               22500     
                                                                 
 batch_normalization_1 (Batc  (None, 150)              600       
 hNormalization)                                                 
                                                                 
 dense_2 (Dense)             (None, 50)                7550

Epoch 48/500
120/120 [==============================] - 1s 7ms/step - loss: 2.9655 - accuracy: 0.7192 - val_loss: 3.0040 - val_accuracy: 0.6984
Epoch 49/500
120/120 [==============================] - 1s 7ms/step - loss: 2.9512 - accuracy: 0.7234 - val_loss: 2.9921 - val_accuracy: 0.7008
Epoch 50/500
120/120 [==============================] - 1s 7ms/step - loss: 2.9400 - accuracy: 0.7268 - val_loss: 2.9804 - val_accuracy: 0.7041
Epoch 51/500
120/120 [==============================] - 1s 7ms/step - loss: 2.9294 - accuracy: 0.7264 - val_loss: 2.9684 - val_accuracy: 0.7065
Epoch 52/500
120/120 [==============================] - 1s 7ms/step - loss: 2.9169 - accuracy: 0.7293 - val_loss: 2.9567 - val_accuracy: 0.7084
Epoch 53/500
120/120 [==============================] - 1s 7ms/step - loss: 2.9030 - accuracy: 0.7350 - val_loss: 2.9454 - val_accuracy: 0.7098
Epoch 54/500
120/120 [==============================] - 1s 7ms/step - loss: 2.8917 - accuracy: 0.7362 - val_loss: 2.9341 - val_accuracy:

Epoch 105/500
120/120 [==============================] - 1s 7ms/step - loss: 2.4019 - accuracy: 0.8108 - val_loss: 2.4684 - val_accuracy: 0.7775
Epoch 106/500
120/120 [==============================] - 1s 7ms/step - loss: 2.3935 - accuracy: 0.8091 - val_loss: 2.4608 - val_accuracy: 0.7778
Epoch 107/500
120/120 [==============================] - 1s 7ms/step - loss: 2.3846 - accuracy: 0.8121 - val_loss: 2.4531 - val_accuracy: 0.7782
Epoch 108/500
120/120 [==============================] - 1s 7ms/step - loss: 2.3751 - accuracy: 0.8136 - val_loss: 2.4457 - val_accuracy: 0.7791
Epoch 109/500
120/120 [==============================] - 1s 7ms/step - loss: 2.3696 - accuracy: 0.8136 - val_loss: 2.4382 - val_accuracy: 0.7794
Epoch 110/500
120/120 [==============================] - 1s 7ms/step - loss: 2.3607 - accuracy: 0.8137 - val_loss: 2.4308 - val_accuracy: 0.7807
Epoch 111/500
120/120 [==============================] - 1s 7ms/step - loss: 2.3531 - accuracy: 0.8154 - val_loss: 2.4233 - val_ac

120/120 [==============================] - 1s 7ms/step - loss: 1.6411 - accuracy: 0.8689 - val_loss: 1.7783 - val_accuracy: 0.8124
Epoch 218/500
120/120 [==============================] - 1s 7ms/step - loss: 1.6351 - accuracy: 0.8690 - val_loss: 1.7730 - val_accuracy: 0.8123
Epoch 219/500
120/120 [==============================] - 1s 7ms/step - loss: 1.6308 - accuracy: 0.8668 - val_loss: 1.7679 - val_accuracy: 0.8127
Epoch 220/500
120/120 [==============================] - 1s 7ms/step - loss: 1.6243 - accuracy: 0.8683 - val_loss: 1.7628 - val_accuracy: 0.8126
Epoch 221/500
120/120 [==============================] - 1s 7ms/step - loss: 1.6214 - accuracy: 0.8682 - val_loss: 1.7580 - val_accuracy: 0.8131
Epoch 222/500
120/120 [==============================] - 1s 7ms/step - loss: 1.6123 - accuracy: 0.8684 - val_loss: 1.7527 - val_accuracy: 0.8137
Epoch 223/500
120/120 [==============================] - 1s 7ms/step - loss: 1.6077 - accuracy: 0.8692 - val_loss: 1.7478 - val_accuracy: 0.8132

120/120 [==============================] - 1s 7ms/step - loss: 1.0587 - accuracy: 0.9064 - val_loss: 1.3006 - val_accuracy: 0.8126
Epoch 330/500
120/120 [==============================] - 1s 7ms/step - loss: 1.0560 - accuracy: 0.9056 - val_loss: 1.2970 - val_accuracy: 0.8126
Epoch 331/500
120/120 [==============================] - 1s 7ms/step - loss: 1.0490 - accuracy: 0.9071 - val_loss: 1.2935 - val_accuracy: 0.8126
Epoch 332/500
120/120 [==============================] - 1s 7ms/step - loss: 1.0446 - accuracy: 0.9073 - val_loss: 1.2908 - val_accuracy: 0.8125
Epoch 333/500
120/120 [==============================] - 1s 7ms/step - loss: 1.0421 - accuracy: 0.9073 - val_loss: 1.2874 - val_accuracy: 0.8124
Epoch 334/500
120/120 [==============================] - 1s 7ms/step - loss: 1.0339 - accuracy: 0.9106 - val_loss: 1.2834 - val_accuracy: 0.8125
Epoch 335/500
120/120 [==============================] - 1s 7ms/step - loss: 1.0306 - accuracy: 0.9090 - val_loss: 1.2799 - val_accuracy: 0.8127

INFO:tensorflow:Assets written to: ./TrainedModels/Trained_Traits_Model_BM.acc.mod/assets


In [10]:
################################################################################################################################################
#Train the combined SNPs + traits (OU) network
################################################################################################################################################

traits_train, traits_test = traits_OU[int(len(y)*.25):], traits_OU[:int(len(y)*.25)]

traits_train=traits_train.reshape((traits_train.shape[0], (traits_train.shape[1]*traits_train.shape[2])))
traits_test=traits_test.reshape((traits_test.shape[0], (traits_test.shape[1]*traits_test.shape[2])))

# Create the MLP and CNN models
mlp = create_mlp(traits_train)

# Create the input to the final set of layers as the output of both the MLP and CNN
combinedInput = LinearW()([mlp.output, cnn.output])

# The final fully-connected layer head will have two dense layers (one relu and one sigmoid)
x = Dense(50, activation="relu")(combinedInput)
x = Dense(num_classes, activation="softmax")(x)


# The final model accepts numerical data on the MLP input and images on the CNN input, outputting a single value
model4 = Model(inputs=[mlp.input, cnn.input], outputs=x)

opt = SGD(learning_rate=0.001)

model4.compile(loss=keras.losses.categorical_crossentropy,
	              optimizer=opt,
	              metrics=['accuracy'])

print(model4.summary())

earlyStopping = EarlyStopping(monitor='val_accuracy', patience=100, verbose=0, mode='max', restore_best_weights=True)
start = time.time()
model4.fit([traits_train, xtrain], ytrain, batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=([traits_test, xtest], ytest),callbacks=[earlyStopping])
print (f'Time: {time.time() - start}')

model4.save(filepath='./TrainedModels/Trained_Comb_Model_1KSNPs_OU.acc.mod')

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 429, 109)]   0           []                               
                                                                                                  
 conv1d (Conv1D)                (None, 427, 250)     81750       ['input_1[0][0]']                
                                                                                                  
 batch_normalization_2 (BatchNo  (None, 427, 250)    1000        ['conv1d[0][0]']                 
 rmalization)                                                                                     
                                                                                                  
 conv1d_1 (Conv1D)              (None, 425, 250)     187500      ['batch_normalization_2[0][

120/120 [==============================] - 6s 53ms/step - loss: 3.4087 - accuracy: 0.9231 - val_loss: 3.2452 - val_accuracy: 0.9867
Epoch 17/100
120/120 [==============================] - 6s 54ms/step - loss: 3.3904 - accuracy: 0.9286 - val_loss: 3.2348 - val_accuracy: 0.9865
Epoch 18/100
120/120 [==============================] - 7s 55ms/step - loss: 3.3659 - accuracy: 0.9349 - val_loss: 3.2236 - val_accuracy: 0.9872
Epoch 19/100
120/120 [==============================] - 6s 53ms/step - loss: 3.3403 - accuracy: 0.9435 - val_loss: 3.2109 - val_accuracy: 0.9876
Epoch 20/100
120/120 [==============================] - 6s 53ms/step - loss: 3.3232 - accuracy: 0.9455 - val_loss: 3.1966 - val_accuracy: 0.9902
Epoch 21/100
120/120 [==============================] - 7s 55ms/step - loss: 3.3034 - accuracy: 0.9506 - val_loss: 3.1904 - val_accuracy: 0.9880
Epoch 22/100
120/120 [==============================] - 7s 54ms/step - loss: 3.2906 - accuracy: 0.9517 - val_loss: 3.1756 - val_accuracy: 0.991

120/120 [==============================] - 6s 51ms/step - loss: 2.8083 - accuracy: 0.9888 - val_loss: 2.7772 - val_accuracy: 0.9984
Epoch 73/100
120/120 [==============================] - 6s 53ms/step - loss: 2.7996 - accuracy: 0.9896 - val_loss: 2.7698 - val_accuracy: 0.9984
Epoch 74/100
120/120 [==============================] - 6s 53ms/step - loss: 2.7944 - accuracy: 0.9891 - val_loss: 2.7630 - val_accuracy: 0.9982
Epoch 75/100
120/120 [==============================] - 6s 52ms/step - loss: 2.7850 - accuracy: 0.9904 - val_loss: 2.7554 - val_accuracy: 0.9982
Epoch 76/100
120/120 [==============================] - 6s 52ms/step - loss: 2.7766 - accuracy: 0.9906 - val_loss: 2.7493 - val_accuracy: 0.9977
Epoch 77/100
120/120 [==============================] - 6s 52ms/step - loss: 2.7695 - accuracy: 0.9902 - val_loss: 2.7410 - val_accuracy: 0.9983
Epoch 78/100
120/120 [==============================] - 6s 52ms/step - loss: 2.7608 - accuracy: 0.9911 - val_loss: 2.7342 - val_accuracy: 0.998

INFO:tensorflow:Assets written to: ./TrainedModels/Trained_Comb_Model_1KSNPs_OU.acc.mod/assets


INFO:tensorflow:Assets written to: ./TrainedModels/Trained_Comb_Model_1KSNPs_OU.acc.mod/assets


In [11]:
################################################################################################################################################
#Train the traits only (OU) network
################################################################################################################################################
xMLP = Dense(num_classes, activation="softmax")(mlp.output)

model5 = Model(inputs=mlp.input, outputs=xMLP)

model5.compile(loss=keras.losses.categorical_crossentropy,
	              optimizer=opt,
	              metrics=['accuracy'])

print(model5.summary())

start = time.time()
model5.fit(traits_train, ytrain, batch_size=batch_size,
          epochs=epochs_traits,
          verbose=1,
          validation_data=(traits_test, ytest),callbacks=[earlyStopping])
print (f'Time: {time.time() - start}')
          
model5.save(filepath='./TrainedModels/Trained_Traits_Model_OU.acc.mod')

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_6_input (InputLayer)  [(None, 436)]             0         
                                                                 
 dense_6 (Dense)             (None, 150)               65400     
                                                                 
 batch_normalization_5 (Batc  (None, 150)              600       
 hNormalization)                                                 
                                                                 
 dense_7 (Dense)             (None, 150)               22500     
                                                                 
 batch_normalization_6 (Batc  (None, 150)              600       
 hNormalization)                                                 
                                                                 
 dense_8 (Dense)             (None, 50)                7550

Epoch 48/500
120/120 [==============================] - 1s 7ms/step - loss: 2.9903 - accuracy: 0.7072 - val_loss: 3.0437 - val_accuracy: 0.6750
Epoch 49/500
120/120 [==============================] - 1s 7ms/step - loss: 2.9789 - accuracy: 0.7090 - val_loss: 3.0318 - val_accuracy: 0.6782
Epoch 50/500
120/120 [==============================] - 1s 7ms/step - loss: 2.9658 - accuracy: 0.7126 - val_loss: 3.0202 - val_accuracy: 0.6816
Epoch 51/500
120/120 [==============================] - 1s 7ms/step - loss: 2.9541 - accuracy: 0.7146 - val_loss: 3.0087 - val_accuracy: 0.6838
Epoch 52/500
120/120 [==============================] - 1s 7ms/step - loss: 2.9434 - accuracy: 0.7171 - val_loss: 2.9972 - val_accuracy: 0.6866
Epoch 53/500
120/120 [==============================] - 1s 7ms/step - loss: 2.9293 - accuracy: 0.7191 - val_loss: 2.9860 - val_accuracy: 0.6891
Epoch 54/500
120/120 [==============================] - 1s 7ms/step - loss: 2.9190 - accuracy: 0.7213 - val_loss: 2.9750 - val_accuracy:

Epoch 105/500
120/120 [==============================] - 1s 7ms/step - loss: 2.4277 - accuracy: 0.7959 - val_loss: 2.5204 - val_accuracy: 0.7535
Epoch 106/500
120/120 [==============================] - 1s 7ms/step - loss: 2.4184 - accuracy: 0.7966 - val_loss: 2.5130 - val_accuracy: 0.7539
Epoch 107/500
120/120 [==============================] - 1s 7ms/step - loss: 2.4108 - accuracy: 0.7975 - val_loss: 2.5056 - val_accuracy: 0.7539
Epoch 108/500
120/120 [==============================] - 1s 7ms/step - loss: 2.4029 - accuracy: 0.7992 - val_loss: 2.4982 - val_accuracy: 0.7551
Epoch 109/500
120/120 [==============================] - 1s 7ms/step - loss: 2.3941 - accuracy: 0.7992 - val_loss: 2.4910 - val_accuracy: 0.7560
Epoch 110/500
120/120 [==============================] - 1s 7ms/step - loss: 2.3877 - accuracy: 0.8001 - val_loss: 2.4836 - val_accuracy: 0.7559
Epoch 111/500
120/120 [==============================] - 1s 7ms/step - loss: 2.3790 - accuracy: 0.8003 - val_loss: 2.4763 - val_ac

120/120 [==============================] - 1s 7ms/step - loss: 2.0154 - accuracy: 0.8295 - val_loss: 2.1506 - val_accuracy: 0.7743
Epoch 162/500
120/120 [==============================] - 1s 7ms/step - loss: 2.0079 - accuracy: 0.8312 - val_loss: 2.1447 - val_accuracy: 0.7744
Epoch 163/500
120/120 [==============================] - 1s 7ms/step - loss: 2.0020 - accuracy: 0.8302 - val_loss: 2.1388 - val_accuracy: 0.7744
Epoch 164/500
120/120 [==============================] - 1s 7ms/step - loss: 1.9940 - accuracy: 0.8339 - val_loss: 2.1329 - val_accuracy: 0.7741
Epoch 165/500
120/120 [==============================] - 1s 7ms/step - loss: 1.9860 - accuracy: 0.8322 - val_loss: 2.1271 - val_accuracy: 0.7746
Epoch 166/500
120/120 [==============================] - 1s 7ms/step - loss: 1.9803 - accuracy: 0.8321 - val_loss: 2.1213 - val_accuracy: 0.7748
Epoch 167/500
120/120 [==============================] - 1s 7ms/step - loss: 1.9732 - accuracy: 0.8348 - val_loss: 2.1155 - val_accuracy: 0.7753

120/120 [==============================] - 1s 7ms/step - loss: 1.6589 - accuracy: 0.8574 - val_loss: 1.8482 - val_accuracy: 0.7822
Epoch 218/500
120/120 [==============================] - 1s 7ms/step - loss: 1.6533 - accuracy: 0.8561 - val_loss: 1.8433 - val_accuracy: 0.7820
Epoch 219/500
120/120 [==============================] - 1s 7ms/step - loss: 1.6471 - accuracy: 0.8557 - val_loss: 1.8385 - val_accuracy: 0.7824
Epoch 220/500
120/120 [==============================] - 1s 7ms/step - loss: 1.6401 - accuracy: 0.8575 - val_loss: 1.8337 - val_accuracy: 0.7825
Epoch 221/500
120/120 [==============================] - 1s 7ms/step - loss: 1.6356 - accuracy: 0.8573 - val_loss: 1.8288 - val_accuracy: 0.7831
Epoch 222/500
120/120 [==============================] - 1s 7ms/step - loss: 1.6304 - accuracy: 0.8601 - val_loss: 1.8239 - val_accuracy: 0.7831
Epoch 223/500
120/120 [==============================] - 1s 7ms/step - loss: 1.6225 - accuracy: 0.8596 - val_loss: 1.8190 - val_accuracy: 0.7837

120/120 [==============================] - 1s 7ms/step - loss: 1.3435 - accuracy: 0.8804 - val_loss: 1.5988 - val_accuracy: 0.7843
Epoch 274/500
120/120 [==============================] - 1s 7ms/step - loss: 1.3399 - accuracy: 0.8804 - val_loss: 1.5949 - val_accuracy: 0.7855
Epoch 275/500
120/120 [==============================] - 1s 7ms/step - loss: 1.3330 - accuracy: 0.8824 - val_loss: 1.5911 - val_accuracy: 0.7849
Epoch 276/500
120/120 [==============================] - 1s 7ms/step - loss: 1.3282 - accuracy: 0.8825 - val_loss: 1.5871 - val_accuracy: 0.7852
Epoch 277/500
120/120 [==============================] - 1s 7ms/step - loss: 1.3217 - accuracy: 0.8836 - val_loss: 1.5831 - val_accuracy: 0.7852
Epoch 278/500
120/120 [==============================] - 1s 7ms/step - loss: 1.3171 - accuracy: 0.8831 - val_loss: 1.5794 - val_accuracy: 0.7853
Epoch 279/500
120/120 [==============================] - 1s 7ms/step - loss: 1.3134 - accuracy: 0.8847 - val_loss: 1.5753 - val_accuracy: 0.7862

120/120 [==============================] - 1s 7ms/step - loss: 1.0620 - accuracy: 0.9080 - val_loss: 1.3974 - val_accuracy: 0.7822
Epoch 330/500
120/120 [==============================] - 1s 7ms/step - loss: 1.0559 - accuracy: 0.9080 - val_loss: 1.3944 - val_accuracy: 0.7828
Epoch 331/500
120/120 [==============================] - 1s 7ms/step - loss: 1.0532 - accuracy: 0.9075 - val_loss: 1.3910 - val_accuracy: 0.7828
Epoch 332/500
120/120 [==============================] - 1s 7ms/step - loss: 1.0475 - accuracy: 0.9079 - val_loss: 1.3883 - val_accuracy: 0.7824
Epoch 333/500
120/120 [==============================] - 1s 7ms/step - loss: 1.0430 - accuracy: 0.9076 - val_loss: 1.3853 - val_accuracy: 0.7827
Epoch 334/500
120/120 [==============================] - 1s 7ms/step - loss: 1.0392 - accuracy: 0.9088 - val_loss: 1.3823 - val_accuracy: 0.7826
Epoch 335/500
120/120 [==============================] - 1s 7ms/step - loss: 1.0340 - accuracy: 0.9075 - val_loss: 1.3793 - val_accuracy: 0.7819

INFO:tensorflow:Assets written to: ./TrainedModels/Trained_Traits_Model_OU.acc.mod/assets


In [12]:
#################################################################################################################################################
##Perform cross-validation
#################################################################################################################################################


model1 = load_model('./TrainedModels/Trained_Comb_Model_1KSNPs_BM.acc.mod')
model2 = load_model('./TrainedModels/Trained_CNN_Model_1KSNPs.acc.mod')
model3 = load_model('./TrainedModels/Trained_Traits_Model_BM.acc.mod')
model4 = load_model('./TrainedModels/Trained_Comb_Model_1KSNPs_OU.acc.mod')
model5 = load_model('./TrainedModels/Trained_Traits_Model_OU.acc.mod')

In [13]:
traits_BM = []
traits_BM = np.loadtxt("./testSims/traits/traits_BM.txt").reshape(4000,-1,4)
traits_BM = np.array(traits_BM)

#This is not behaving well. I need to use another scaling scheme.
for i in range(traits_BM.shape[2]):
    traits_BM[:, :, i] = scalers_BM[i].transform(traits_BM[:, :, i]) 

traits_BM = np.array(traits_BM)

traits_OU = []
traits_OU = np.loadtxt("./testSims/traits/traits_OU.txt").reshape(4000,-1,4)
traits_OU = np.array(traits_OU)

for i in range(traits_OU.shape[2]):
    traits_OU[:, :, i] = scalers_OU[i].transform(traits_OU[:, :, i]) 

traits_OU = np.array(traits_OU)

#Add missing individuals in E. balsamifera subsp. balsamifera as 0s
for i in range(traits_BM.shape[0]):
  for m in range(51):
    j = random.randint(19, 98)
    traits_BM[i,j,:] = 0
    
#Add missing individuals in E. balsamifera subsp. balsamifera as 0s
for i in range(traits_OU.shape[0]):
  for m in range(51):
    j = random.randint(19, 98)
    traits_OU[i,j,:] = 0

u1 = np.load("./testSims/Model_1sp.npz")
u2 = np.load("./testSims/Model_2spMorph.npz")
u3 = np.load("./testSims/Model_2spPhylo.npz")
u4 = np.load("./testSims/Model_3sp.npz")

xtest=np.concatenate((u1['Model_1sp'],u2['Model_2spMorph'],u3['Model_2spPhylo'],u4['Model_3sp']),axis=0)

#transform major alleles in -1 and minor 1
for arr,array in enumerate(xtest):
  for idx,row in enumerate(array):
    if np.count_nonzero(row) > len(row)/2:
      xtest[arr][idx][xtest[arr][idx] == 1] = -1
      xtest[arr][idx][xtest[arr][idx] == 0] = 1
    else:
      xtest[arr][idx][xtest[arr][idx] == 0] = -1

#Add missing data as 0s, according to a specifies missing data percentage
#46,761 SNP datapoints and 8,287 missing genotypes = 17.7%
missD_perc = 17.7
missD = int(xtest.shape[1]*xtest.shape[2]*(missD_perc/100))
for i in range(xtest.shape[0]):
  for m in range(missD):
    j = random.randint(0, xtest.shape[1] - 1)
    k = random.randint(0, xtest.shape[2] - 1)
    xtest[i][j][k] = 0


ytest=[0 for i in range(len(u1['Model_1sp']))]
ytest.extend([1 for i in range(len(u2['Model_2spMorph']))])
ytest.extend([2 for i in range(len(u3['Model_2spPhylo']))])
ytest.extend([3 for i in range(len(u4['Model_3sp']))])
ytest = np.array(ytest)

In [14]:
#first get the predictions
pred = model1.predict([traits_BM.reshape(4000,-1), xtest])
pred_cat = [i.argmax() for i in pred]
ytest_cat = [i.argmax() for i in ytest]
print (confusion_matrix(ytest, pred_cat))


#first get the predictions
pred = model2.predict(xtest)
pred_cat = [i.argmax() for i in pred]
ytest_cat = [i.argmax() for i in ytest]
print (confusion_matrix(ytest, pred_cat))

#first get the predictions
pred = model3.predict(traits_BM.reshape(4000,-1))
pred_cat = [i.argmax() for i in pred]
ytest_cat = [i.argmax() for i in ytest]
print (confusion_matrix(ytest, pred_cat))

#first get the predictions
pred = model4.predict([traits_OU.reshape(4000,-1), xtest])
pred_cat = [i.argmax() for i in pred]
ytest_cat = [i.argmax() for i in ytest]
print (confusion_matrix(ytest, pred_cat))

#first get the predictions
pred = model5.predict(traits_OU.reshape(4000,-1))
pred_cat = [i.argmax() for i in pred]
ytest_cat = [i.argmax() for i in ytest]
print (confusion_matrix(ytest, pred_cat))

125/125 [==============================] - 1s 6ms/step
[[1000    0    0    0]
 [   0 1000    0    0]
 [   2    0  997    1]
 [   0    1    4  995]]
125/125 [==============================] - 1s 5ms/step
[[1000    0    0    0]
 [   0 1000    0    0]
 [   2    0  998    0]
 [   0    1    3  996]]
125/125 [==============================] - 0s 3ms/step
[[877  36  66  21]
 [ 51 892   8  49]
 [ 88   9 821  82]
 [ 41 109 169 681]]
125/125 [==============================] - 1s 5ms/step
[[1000    0    0    0]
 [   0 1000    0    0]
 [   4    0  996    0]
 [   1    1    3  995]]
125/125 [==============================] - 0s 3ms/step
[[861  34  77  28]
 [ 56 856   7  81]
 [115  16 768 101]
 [ 43 113 170 674]]


In [16]:
################################################################################################################################################
#Predict empirical data
################################################################################################################################################

inSNPs=np.loadtxt("./input_SNPs.txt")
EmpSNPs = np.array(inSNPs)

#transform major alleles in -1 and minor 1
for idx,row in enumerate(EmpSNPs):
  if np.count_nonzero(row==1) > np.count_nonzero(row==-1):
    EmpSNPs[idx][EmpSNPs[idx] == 1] = 9
    EmpSNPs[idx][EmpSNPs[idx] == -1] = 1
    EmpSNPs[idx][EmpSNPs[idx] == 9] = -1

EmpSNPs = np.repeat(EmpSNPs[np.newaxis, :, :], 100, axis=0)

ade=np.genfromtxt("./input_traits_ade.txt", delimiter="\t", filling_values=0)
bal=np.genfromtxt("./input_traits_bal.txt", delimiter="\t", filling_values=0)
sep=np.genfromtxt("./input_traits_sep.txt", delimiter="\t", filling_values=0)

ade=np.array(ade)
bal=np.array(bal)
sep=np.array(sep)

res = []
for i in range(0,100):
  idx_ade = np.random.choice(ade.shape[0], 19, replace=False)
  n = ade[idx_ade,:]
  idx_bal = np.random.choice(bal.shape[0], 29, replace=False)
  n_z = np.zeros((80,4))
  for i in range(len(idx_bal)):
    idx_nz=np.random.choice(80, len(idx_bal), replace=False)
    n_z[idx_nz[i]] = bal[idx_bal[i],:]
  n = np.concatenate((n,n_z), axis=0)
  idx_sep = np.random.choice(sep.shape[0], 10, replace=False)
  n = np.concatenate((n,sep[idx_sep,:]), axis=0)
  res.append(np.array(n))

traits = np.array(res)

for i in range(traits.shape[2]):
    traits[:, :, i] = scalers_BM[i].transform(traits[:, :, i])

emp_traits_BM = np.array(traits)

traits = np.array(res)

for i in range(traits.shape[2]):
    traits[:, :, i] = scalers_OU[i].transform(traits[:, :, i])

emp_traits_OU = np.array(traits)

In [17]:
Emp_CombBM_pred = model1.predict([emp_traits_BM.reshape(100,-1),EmpSNPs])

np.savetxt("Pred_Emp_Comb_BM_Predictions.txt", Emp_CombBM_pred)

Emp_SNP_pred = model2.predict(EmpSNPs)

np.savetxt("Pred_Emp_SNP_Predictions.txt", Emp_SNP_pred)

Emp_traitsBM_pred = model3.predict(emp_traits_BM.reshape(100,-1))

np.savetxt("Pred_Emp_traits_BM_Predictions.txt", Emp_traitsBM_pred)

Emp_CombOU_pred = model4.predict([emp_traits_OU.reshape(100,-1),EmpSNPs])

np.savetxt("Pred_Emp_Comb_OU_Predictions.txt", Emp_CombOU_pred)

Emp_traitsOU_pred = model5.predict(emp_traits_OU.reshape(100,-1))

np.savetxt("Pred_Emp_traits_OU_Predictions.txt", Emp_traitsOU_pred)

4/4 [==============================] - 0s 2ms/step
